In [1]:
"""
< 앙상블 학습모델 >
 - 결정트리를 기반으로 만들어진 모델들
 - 기존 하나의 결정트리를 사용했다면,
   앙상블 모델은 여러개의 결정트리를 사용한다고 이해..
 
< 앙상블 모델 >
 ( 분류 및 회귀 모두 사용가능)
  - 랜덤포레스트(RandomForest)
  - 엑스트라트리(Extra Trees)
  - 그래디언트 부스팅(Gradient Boosting)
  - 히스토그램 기반 그래디언트 부스팅(Historgram-base Gradient Boosting)
 
 (분류로만 사용가능)
  - XGBoosting
  - LightGBM
"""

'\n< 앙상블 학습모델 >\n - 결정트리를 기반으로 만들어진 모델들\n - 기존 하나의 결정트리를 사용했다면,\n   앙상블 모델은 여러개의 결정트리를 사용한다고 이해..\n \n< 앙상블 모델 >\n ( 분류 및 회귀 모두 사용가능)\n  - 랜덤포레스트(RandomForest)\n  - 엑스트라트리(Extra Trees)\n  - 그래디언트 부스팅(Gradient Boosting)\n  - 히스토그램 기반 그래디언트 부스팅(Historgram-base Gradient Boosting)\n \n (분류로만 사용가능)\n  - XGBoosting\n  - LightGBM\n'

### 랜덤포레스트(RandomForest)

In [2]:
"""
- 앙상블 학습 모델 중 가장 대표격 모델
- 가장 먼저 시작하는 모델로 사용됨
- 훈련데이터에서 과대적합되는 것을 막아줍니다.
- 검증데이터와 테스트데이터에서 안정적인 성능을 얻을 수 있음

< 학습개념 >
 - 결정트리 여러개를 이용함
 - 여러개의 결정트리를 랜덤하게 만듬
 - 훈련시 랜덤하게 훈련데이터 샘플을 추출
 - 사용된 훈련데이터 샘플을 반환하여 다시 랜덤하게 훈련데이터를 추출하여 훈련
 
< 훈련 데이터 사용순서 >
 - 데이터 원본에서 중복을 허용하여 데이터를 샘플링함
 - 원본에서 랜덤하게 샘플 추출
 - 사용이 끝난 샘플을 원본에 반환
 - 다시 랜덤하게 샘플을 추출
 - 원본에서 랜덤하게 샘플을 반환하면서 추출하기에
   훈련에 사용되었던 값들이 다시 추출될 수 있음
 ==> 이러한 훈련 데이터 흐름을 "부트스트랩 샘플링" 이라고 합니다.
"""

'\n- 앙상블 학습 모델 중 가장 대표격 모델\n- 가장 먼저 시작하는 모델로 사용됨\n- 훈련데이터에서 과대적합되는 것을 막아줍니다.\n- 검증데이터와 테스트데이터에서 안정적인 성능을 얻을 수 있음\n\n< 학습개념 >\n - 결정트리 여러개를 이용함\n - 여러개의 결정트리를 랜덤하게 만듬\n - 훈련시 랜덤하게 훈련데이터 샘플을 추출\n - 사용된 훈련데이터 샘플을 반환하여 다시 랜덤하게 훈련데이터를 추출하여 훈련\n \n< 훈련 데이터 사용순서 >\n - 데이터 원본에서 중복을 허용하여 데이터를 샘플링함\n - 원본에서 랜덤하게 샘플 추출\n - 사용이 끝난 샘플을 원본에 반환\n - 다시 랜덤하게 샘플을 추출\n - 원본에서 랜덤하게 샘플을 반환하면서 추출하기에\n   훈련에 사용되었던 값들이 다시 추출될 수 있음\n ==> 이러한 훈련 데이터 흐름을 "부트스트랩 샘플링" 이라고 합니다.\n'

In [3]:
import pandas as pd
import numpy as np

In [4]:
file_path = "./data/08_wine.csv"
wine = pd.read_csv(file_path)
wine

### 독립 및 종속변수 추출
data = wine[["alcohol", "sugar", "pH"]].to_numpy()
target = wine["class"].to_numpy()

data.shape, target.shape

((6497, 3), (6497,))

In [5]:
### 훈련 및 테스트 데이터 분리하기
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = \
              train_test_split(data, target, test_size= 0.2, random_state=42)

print(train_input.shape, train_target.shape)
print(test_input.shape, test_target.shape)

(5197, 3) (5197,)
(1300, 3) (1300,)


### 랜덤포레스트 훈련 정확도 확인하기

In [6]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

In [7]:
### 클래스(모델) 생성하기
# n_jobs : 사용할 CPU core 갯수 (-1은 모든 코어 사용)
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
rf

RandomForestClassifier(n_jobs=-1, random_state=42)

In [8]:
### 교차검증으로 훈련시키기
# - return_train_score : 훈련정확도를 포함해서 조회하고자 할때 True 사용
scores = cross_validate(rf, train_input, train_target, n_jobs=-1, return_train_score=True)
scores

{'fit_time': array([0.15150476, 0.16929197, 0.15366793, 0.20053768, 0.15366793]),
 'score_time': array([0.0177865 , 0.01562309, 0.01562405, 0.01562333, 0.01562405]),
 'test_score': array([0.88461538, 0.88942308, 0.90279115, 0.88931665, 0.88642926]),
 'train_score': array([0.9971133 , 0.99663219, 0.9978355 , 0.9973545 , 0.9978355 ])}

In [9]:
### 훈련 정확도와 테스트 정확도 확인하기
np.mean(scores["train_score"]), np.mean(scores["test_score"])

(0.9973541965122431, 0.8905151032797809)

In [10]:
### 랜덤포레스트는 특성들 중 영향을 많이 미친 값들을 비교할 수 있음
# - 특성중요도라고 칭함 : feature_importances_
rf.fit(train_input, train_target)
rf.feature_importances_

array([0.23167441, 0.50039841, 0.26792718])

In [11]:
### 랜덤포레스트 특징
# - 훈련데이터를 랜덤하게 추출해서 사용하다보면
#   훈련에 참여하지 못하는 값들이 있을 수 있음
# - 이처럼 훈련에 참여하지 못한 값들을 이용해서 다시 검증을 할 수 있음
# - 이때 사용하는 방법 : oob (Out of Bag) 이라는 개념을 적용
rf = RandomForestClassifier(n_jobs=-1, oob_score=True, random_state=42)
rf.fit(train_input, train_target)

RandomForestClassifier(n_jobs=-1, oob_score=True, random_state=42)

In [12]:
### oob_score 정확도 확인
# - 검증용 데이터로 사용됨
rf.oob_score_

0.8934000384837406

In [51]:
# --> 교차검증없이 모델 수행해봄 --> 이건 내가 그냥 해봄
rf.fit(train_input, train_target)
rf.score(train_input, train_target), rf.score(test_input, test_target)

(0.996921300750433, 0.8892307692307693)

# 엑스트라 트리(Extra Trees)

In [13]:
"""
- 랜덤포레스트와 유사하게 작동됨
- 기본적으로 100개의 결정트리로 훈련됨

< 랜덤포레스트와의 차이점 >
 - 랜덤포레스트는 부트스트랩 샘플링 기법을 사용하여 중복을 허용하여 샘플링
 - 엑스트라트리는 부트스트랩을 지원하지 않으며,
   훈련데이터 전체를 이용하여 결정트리를 생성함
 - 무작위로 트리를 분할함
 
< 특징 >
 - 과대적합을 막고, 검증데이터의 정확도를 높일 수 있음
 - 다만, 특성이 많이 않은 경우 랜덤포레스트와 큰 차이 없음
 - 랜덤포레스트보다 무작위성이 크기 때문에,
   더 많은 결정트리를 훈련시킴
 - 랜덤하게 트리를 분할하기 때문에 계산 속도가 빠른 장정이 있음  
"""

'\n- 랜덤포레스트와 유사하게 작동됨\n- 기본적으로 100개의 결정트리로 훈련됨\n\n< 랜덤포레스트와의 차이점 >\n - 랜덤포레스트는 부트스트랩 샘플링 기법을 사용하여 중복을 허용하여 샘플링\n - 엑스트라트리는 부트스트랩을 지원하지 않으며,\n   훈련데이터 전체를 이용하여 결정트리를 생성함\n - 무작위로 트리를 분할함\n \n< 특징 >\n - 과대적합을 막고, 검증데이터의 정확도를 높일 수 있음\n - 다만, 특성이 많이 않은 경우 랜덤포레스트와 큰 차이 없음\n - 랜덤포레스트보다 무작위성이 크기 때문에,\n   더 많은 결정트리를 훈련시킴\n - 랜덤하게 트리를 분할하기 때문에 계산 속도가 빠른 장정이 있음  \n'

In [14]:
from sklearn.ensemble import ExtraTreesClassifier

In [15]:
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
et

ExtraTreesClassifier(n_jobs=-1, random_state=42)

In [16]:
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
scores

{'fit_time': array([0.12498474, 0.12498498, 0.14060807, 0.14060903, 0.14060903]),
 'score_time': array([0.03124642, 0.03124595, 0.01562285, 0.03124714, 0.01562357]),
 'test_score': array([0.88365385, 0.87884615, 0.90375361, 0.88835419, 0.88931665]),
 'train_score': array([0.9971133 , 0.99663219, 0.998076  , 0.997595  , 0.9978355 ])}

In [17]:
### 훈련 정확도와 테스트 정확도 확인하기
np.mean(scores["train_score"]), np.mean(scores["test_score"])

(0.9974503966084433, 0.8887848893166506)

In [18]:
### 특성중요도 확인하기
et.fit(train_input, train_target)
et.feature_importances_

array([0.20183568, 0.52242907, 0.27573525])

# 그레디언트 부스팅(Gradient Boosting)

In [19]:
"""
- 트리의 깊이가 얕은 결정트리 사용 (기본적으로 max_depth는 3을 사용)
- 트리의 갯수는 100개를 사용
- 기존에 다른 훈련모델의 결과가 좋지 않을 경우에 사용해 보면 됩니다.
  (또는 훈련모델들의 비교대상으로 사용됩니다.)
- 과대적합 해소에 강하며, 일반화(과대/과소 최적화)에 강함

< 성능향상 테스트 방법 >
 - 결정트리의 갯수를 늘려가면서 훈련테스트
 - 학습률을 지원하기 때문에, 학습률의 값을 증가시켜본다(기본값=0.1)
 
< 단점 >
 - 순서대로 트리를 추가함(랜덤하지 않음)
 - 랜덤하지 않기 때문에, 속도가 다소 느린 단점이 있음
 - 느린 단점을 보완한 모델이 "히스토그램 기반 그레디언트 부스팅" 모델임
"""

'\n- 트리의 깊이가 얕은 결정트리 사용 (기본적으로 max_depth는 3을 사용)\n- 트리의 갯수는 100개를 사용\n- 기존에 다른 훈련모델의 결과가 좋지 않을 경우에 사용해 보면 됩니다.\n  (또는 훈련모델들의 비교대상으로 사용됩니다.)\n- 과대적합 해소에 강하며, 일반화(과대/과소 최적화)에 강함\n\n< 성능향상 테스트 방법 >\n - 결정트리의 갯수를 늘려가면서 훈련테스트\n - 학습률을 지원하기 때문에, 학습률의 값을 증가시켜본다(기본값=0.1)\n \n< 단점 >\n - 순서대로 트리를 추가함(랜덤하지 않음)\n - 랜덤하지 않기 때문에, 속도가 다소 느린 단점이 있음\n - 느린 단점을 보완한 모델이 "히스토그램 기반 그레디언트 부스팅" 모델임\n'

In [20]:
from sklearn.ensemble import GradientBoostingClassifier

In [21]:
### 클래스(모델) 생성하기
gb = GradientBoostingClassifier(random_state=42)
gb

GradientBoostingClassifier(random_state=42)

In [22]:
### 교차검증하기
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
scores

{'fit_time': array([0.17181921, 0.18744254, 0.22416234, 0.22416234, 0.22416234]),
 'score_time': array([0., 0., 0., 0., 0.]),
 'test_score': array([0.86634615, 0.87019231, 0.89412897, 0.86044273, 0.86910491]),
 'train_score': array([0.89006495, 0.88958383, 0.88239538, 0.89249639, 0.88600289])}

In [23]:
### 훈련 정확도와 테스트 정확도 확인하기
np.mean(scores["train_score"]), np.mean(scores["test_score"])

(0.8881086892152563, 0.8720430147331015)

### 훈련 향상시켜보기 : 결정트리 갯수 조정

In [24]:
### 결정트리를 500개로
# - 학습률은 0.2로

In [25]:
# n_estimators : 결정트리 갯수
# learning_rate : 학습률 (높을수록 복잡도를 높이게 됨, 기본=0.1)
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
gb

GradientBoostingClassifier(learning_rate=0.2, n_estimators=500, random_state=42)

In [26]:
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
scores

{'fit_time': array([0.97059727, 1.04705548, 0.98622298, 1.04705548, 1.0418191 ]),
 'score_time': array([0.        , 0.        , 0.01562023, 0.        , 0.00523639]),
 'test_score': array([0.875     , 0.87211538, 0.89701636, 0.8719923 , 0.87391723]),
 'train_score': array([0.9494828 , 0.94443108, 0.94468494, 0.94324194, 0.95045695])}

In [27]:
np.mean(scores["train_score"]), np.mean(scores["test_score"])

(0.9464595437171814, 0.8780082549788999)

In [28]:
### 특성중요도
gb.fit(train_input, train_target)
gb.feature_importances_

array([0.15853457, 0.68010884, 0.1613566 ])

# 히스토그램 기반 그레디언트 부스팅

In [29]:
"""
- 그레디언트 부스팅 모델의 느린 속도를 개선한 모델
- 정형 데이터를 다루는 머신러닝 모델 중 가장 인기가 높음

< 속도가 빠른 이유 >
 - 최초에 특성을 미리 나누어 놓고 시작함
 - 트리를 불할 할 때 매우 빠르게 찾을 수 있음
 
< 성능 향상 방법 >
 - max-iter 매개변수의 값을 변경하여 테스트 진행
 - 기본적으로 기본값을 이용해도 안정적인 성능을 얻을 수 있음
"""

'\n- 그레디언트 부스팅 모델의 느린 속도를 개선한 모델\n- 정형 데이터를 다루는 머신러닝 모델 중 가장 인기가 높음\n\n< 속도가 빠른 이유 >\n - 최초에 특성을 미리 나누어 놓고 시작함\n - 트리를 불할 할 때 매우 빠르게 찾을 수 있음\n \n< 성능 향상 방법 >\n - max-iter 매개변수의 값을 변경하여 테스트 진행\n - 기본적으로 기본값을 이용해도 안정적인 성능을 얻을 수 있음\n'

In [30]:
from sklearn.ensemble import HistGradientBoostingClassifier

In [31]:
### 클래스(모델) 생성하기
hgb = HistGradientBoostingClassifier(random_state=42)
hgb

HistGradientBoostingClassifier(random_state=42)

In [32]:
### 교차검증
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)
scores

{'fit_time': array([0.16176844, 0.14614606, 0.13052106, 0.13052106, 0.16176844]),
 'score_time': array([0.0156579, 0.       , 0.015625 , 0.015625 , 0.0156579]),
 'test_score': array([0.87115385, 0.88365385, 0.90279115, 0.86621752, 0.87680462]),
 'train_score': array([0.93192206, 0.93216262, 0.92857143, 0.93265993, 0.93554594])}

In [33]:
np.mean(scores["train_score"]), np.mean(scores["test_score"])

(0.9321723946453317, 0.8801241948619236)

In [34]:
### 특성중요도
hgb.fit(train_input, train_target)
# --> 이 모델은 변수 중요도 확인 못함

HistGradientBoostingClassifier(random_state=42)

In [35]:
### 예측하기
hgb.predict(test_input)

array([1., 0., 1., ..., 1., 1., 1.])

In [36]:
### 테스트 정확도
hgb.score(test_input, test_target)

0.8723076923076923

In [37]:
### 정확도 값의 비교
# 훈련정확도 > 검증정확도 > 테스트정확도

# 사이킷런 이외 그레디언트 부스팅을 지원하는 패키지

# XGBoost

In [38]:
### 설치해서 사용해야 함
# - conda install -c conda-forge xgboost
# - 위 명령으로 설치 안될 경우 아래 pip로 설치
#   pip install xgboost
from xgboost import XGBClassifier

In [39]:
### 클래스(모델) 생성
# - XGBoost는 사용할 모델을 지정해야함
# - tree_method : 모델 지정(hist : 히스토그램 기반 그레디언트부스팅)
xgb = XGBClassifier(tree_method="hist", random_state=42)
xgb

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=42, ...)

In [40]:
### 교차검증
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)
scores

{'fit_time': array([0.14061022, 0.06249237, 0.14061022, 0.14061022, 0.06249237]),
 'score_time': array([0.        , 0.01565766, 0.        , 0.        , 0.01565766]),
 'test_score': array([0.8625    , 0.8875    , 0.89797883, 0.87969201, 0.8719923 ]),
 'train_score': array([0.95814289, 0.9569401 , 0.95526696, 0.95286195, 0.95430495])}

In [41]:
np.mean(scores["train_score"]), np.mean(scores["test_score"])

(0.9555033709953124, 0.8799326275264677)

# LightGBM

In [42]:
"""
- 마이크로소프트에서 만든 히스토그램 기반 그레디언트 부스트 라이브러리
- 훈련 속도 빠름
- 최신 기술을 많이 적용하고 있어서 인기가 늘고 있는 모델임
- 사이킷런의 히스토그램 기반 그레디언트 부스팅 라이브러리가
  LightGBM의 영향을 받아서 만들어졌다는 설이 있음
"""

'\n- 마이크로소프트에서 만든 히스토그램 기반 그레디언트 부스트 라이브러리\n- 훈련 속도 빠름\n- 최신 기술을 많이 적용하고 있어서 인기가 늘고 있는 모델임\n- 사이킷런의 히스토그램 기반 그레디언트 부스팅 라이브러리가\n  LightGBM의 영향을 받아서 만들어졌다는 설이 있음\n'

In [43]:
### 설치하기
# conda install -c conda-forge lightgbm
# 또는 pip install lightgbm
from lightgbm import LGBMClassifier

In [44]:
### 클래스(모델) 생성하기
lgb = LGBMClassifier(random_state=42)
lgb

LGBMClassifier(random_state=42)

In [47]:
### 교차검증
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)
scores

{'fit_time': array([0.04687071, 0.04687071, 0.03124619, 0.03124619, 0.03124619]),
 'score_time': array([0.0156219 , 0.0156219 , 0.        , 0.01562452, 0.        ]),
 'test_score': array([0.87307692, 0.87692308, 0.91049086, 0.86044273, 0.87969201]),
 'train_score': array([0.93841713, 0.93432764, 0.93410293, 0.93434343, 0.93795094])}

In [48]:
np.mean(scores["train_score"]), np.mean(scores["test_score"])

(0.935828414851749, 0.8801251203079884)

In [50]:
# --> 교차검증없이 모델 수행해봄 --> 이건 내가 그냥 해봄
lgb.fit(train_input, train_target)
lgb.score(train_input, train_target), lgb.score(test_input, test_target)

(0.9288050798537618, 0.8730769230769231)